# MNIST Tasks

This notebook utilizes a simple neural network containing a single n-task context layer to perform multiple tasks using the MNIST dataset

In [1]:
# Import Tensorflow and Keras layers
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout
from tensorflow.keras import Model

import math
import numpy as np
import random

# Utility functions
from utils import idx_load

# Import n-task library
from ntask.atr   import AtrMovingAverage
from ntask.layer import Context
from ntask.model import NTaskModel

## Dataset

In [3]:
# Training images
training_images = idx_load("datasets/mnist/train-images.idx3-ubyte")
training_images.shape

(60000, 28, 28)

In [4]:
# Training labels
training_labels = idx_load("datasets/mnist/train-labels.idx1-ubyte")
training_labels.shape

(60000,)

In [5]:
# Testing images
testing_images = idx_load("datasets/mnist/t10k-images.idx3-ubyte")
testing_images.shape

(10000, 28, 28)

In [6]:
# Testing labels
testing_labels = idx_load("datasets/mnist/t10k-labels.idx1-ubyte")
testing_labels.shape

(10000,)

In [7]:
ds_train = training_images / 255.0

In [8]:
ds_test = testing_images / 255.0

### Loss Function

In [9]:
def loss_fn(labels, predictions):
    return tf.keras.losses.binary_crossentropy(y_true=labels, y_pred=predictions)

### Optimizer

In [10]:
optimizer = tf.keras.optimizers.Adam(1e-4)

## Simple MNIST

### Model

In [16]:
inp = Input((28,28))
x = Flatten()(inp)
x = Dense(128, activation="relu")(x)
x = Dense(10, activation="softmax")(x)

In [17]:
model = Model(inputs=inp, outputs=x)

In [18]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    metrics=['accuracy']
)

In [19]:
model.fit(ds_train, training_labels, epochs=6)

Epoch 1/6
1875/1875 [==============================] - 4s 2ms/step - loss: 1.6229 - accuracy: 0.8827
Epoch 2/6
1875/1875 [==============================] - 3s 2ms/step - loss: 1.5612 - accuracy: 0.9198
Epoch 3/6
1875/1875 [==============================] - 3s 2ms/step - loss: 1.5473 - accuracy: 0.9293
Epoch 4/6
1875/1875 [==============================] - 3s 2ms/step - loss: 1.5382 - accuracy: 0.9361
Epoch 5/6
1875/1875 [==============================] - 4s 2ms/step - loss: 1.5314 - accuracy: 0.9419
Epoch 6/6
1875/1875 [==============================] - 4s 2ms/step - loss: 1.5258 - accuracy: 0.9456


## Model

Waiting on reorganization of dataset handling in NTask model...

In [15]:
inp = Input((28,28))
x = Flatten()(inp)
x = Dense(40)(x)
x = Context()

TypeError: __init__() missing 1 required positional argument: 'atr_model'

In [ ]:
model = NTaskModel(inputs=inp, outputs=x, loss_fn=loss_fn, optimizer=optimizer)